In [2]:
import sys
sys.path.append('/home/austinwang/austin_big_vision')

import io
import jax
import numpy as np
import big_vision.utils as u
from gemma import params as params_lib

path_checkpoint = '/home/austinwang/gemma_ckpt/2b/'
parameters = params_lib.load_and_format_params(path_checkpoint)

local_np_save_path = '/home/austinwang/gemma2b.npz'

2024-08-13 19:19:39.602161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 19:19:39.625370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 19:19:39.632487: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 19:19:40.779464: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
def gemma_pyloop_to_scan(params_pyloop):
  """Converts a python for-loop gemma checkpoint to a lax.scan based one."""
  # On a high level, they are the same except that the for loop has separate
  # array pytrees for each encoderblock, while the scan one has just one
  # encoderblock pytree, with all block's params concatenated.

  params_scan = jax.tree.map(lambda x: x, params_pyloop)  # Structural copy
  t = params_scan["transformer"]

  # Find highest index of layers in the checkpoint (they start at 0):
  layers = {k for k in t if k.startswith("layer_")}
  depth = 1 + max({int(k.split("_")[-1]) for k in layers})

  def stack(*values): return np.stack(values)

  # Stack all layers into a single one:
  t["layer"] = jax.tree.map(stack, *[t[f"layer_{lyr}"] for lyr in range(depth)])

  for lyr in range(depth): del t[f"layer_{lyr}"]

  return params_scan


In [6]:
ckpt = {'params': gemma_pyloop_to_scan(parameters)['transformer']}
names_and_vals, _ = u.tree_flatten_with_names(ckpt)
names_and_vals = [(n, v.astype(np.float32)) for n, v in names_and_vals] # convert bf16 pos_emb to fp32 to avoid void np.dtype error
io_buffer = io.BytesIO()
np.savez(io_buffer, **{k: v for k, v in names_and_vals})
# save to local
with open(local_np_save_path, "wb") as f: f.write(io_buffer.getvalue())

In [7]:
model_p = u.npload(local_np_save_path)
for key in model_p.keys():
    print(key, model_p[key].shape, model_p[key].dtype)

params/embedder/input_embedding (256128, 2048) float32
params/final_norm/scale (2048,) float32
params/layer/attn/attn_vec_einsum/w (18, 8, 256, 2048) float32
params/layer/attn/kv_einsum/w (18, 2, 1, 2048, 256) float32
params/layer/attn/q_einsum/w (18, 8, 2048, 256) float32
params/layer/mlp/gating_einsum (18, 2, 2048, 16384) float32
params/layer/mlp/linear (18, 16384, 2048) float32
params/layer/pre_attention_norm/scale (18, 2048) float32
params/layer/pre_ffw_norm/scale (18, 2048) float32
